In [ ]:
import os
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display

csv_file = 'REFERENCE-v3.csv'
mat_folder = 'training2017'

df = pd.read_csv(csv_file, header=None, names=['FileName', 'Label'])

df = df[~df['Label'].str.contains('~')]

def save_spectrogram(signal, sr, file_name, label, output_dir='spectrograms'):
    label_dir = os.path.join(output_dir, label)
    os.makedirs(label_dir, exist_ok=True)
    
    signal = signal.astype(float)
    
    plt.figure(figsize=(10, 4))
    S = librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=128)
    S_db = librosa.power_to_db(S, ref=np.max)
    
    librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Spectrogram of {file_name}')
    
    save_path = os.path.join(label_dir, f'{file_name}.png')
    plt.savefig(save_path)
    plt.close()

for index, row in df.iterrows():
    mat_file = row['FileName']  
    label = row['Label']        
    
    mat_path = os.path.join(mat_folder, mat_file + '.mat')
    
    if os.path.exists(mat_path):
        mat_data = scipy.io.loadmat(mat_path)
        
        signal = mat_data['val'][0]
        sr = 1000 
        
        save_spectrogram(signal, sr, mat_file, label)
    else:
        print(f'{mat_file} not found!')

print("Spectrogram generation complete.")


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.config.experimental import list_physical_devices, set_memory_growth

# Enable GPU and check if CUDA is available
gpus = list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to prevent TensorFlow from consuming all GPU memory
        for gpu in gpus:
            set_memory_growth(gpu, True)
        print("CUDA is available. Using GPU for training.")
    except RuntimeError as e:
        print(f"Error: {e}")
else:
    print("CUDA is not available. Using CPU for training.")

# Set paths to the spectrogram folders
spectrogram_dir = 'spectrograms'  # This directory contains subfolders for each label (class)
img_size = (128, 128)  # Resize all images to 128x128 (adjust based on your image sizes)
batch_size = 32

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values between 0 and 1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Reserve 20% of data for validation
)

# Limiting the number of images from each folder to 200
# Helper function to limit images from each class
def filter_classes(generator, class_limit=2000):
    filtered_files = []
    for label, files in generator.class_indices.items():
        files = [f for f in generator.filenames if label in f]
        # Take only first 200 images from each class
        filtered_files += files[:class_limit]
    return filtered_files

# Load training and validation data from the spectrogram directories with limits
train_generator = train_datagen.flow_from_directory(
    spectrogram_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    subset='training',  # Train data
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    spectrogram_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    subset='validation'  # Validation data
)

# Manually filter and limit the number of images per class
train_generator.filenames = filter_classes(train_generator, class_limit=200)
validation_generator.filenames = filter_classes(validation_generator, class_limit=200)

# Adjust the number of samples in train and validation generator
train_generator.samples = len(train_generator.filenames)
validation_generator.samples = len(validation_generator.filenames)

# Build the CNN model
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout for regularization
model.add(layers.Dense(3, activation='softmax'))  # 3 output classes

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class classification
              metrics=['accuracy'])

model.summary()

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy:.4f}')


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
from scipy.signal import morlet

def fast_cwt(signal, scales, wavelet_func=morlet):
    """
    Fast Continuous Wavelet Transform (CWT) using FFT.
    
    Args:
    - signal: Input signal (1D numpy array).
    - scales: Array of scales at which to compute the CWT.
    - wavelet_func: Function to generate the wavelet. Default is Morlet.
    
    Returns:
    - cwt_matrix: Matrix of CWT coefficients (2D numpy array).
    """
    n = len(signal)
    signal_fft = fft(signal)  # Compute the FFT of the input signal
    cwt_matrix = np.zeros((len(scales), n), dtype=complex)

    for i, scale in enumerate(scales):
        # Generate the wavelet in frequency domain
        wavelet = wavelet_func(n, w=5.0)  # Morlet wavelet with default w=5
        wavelet_scaled = wavelet * scale

        # FFT of the wavelet
        wavelet_fft = fft(wavelet_scaled, n=n)

        # Convolution in frequency domain (element-wise multiplication)
        conv_result = signal_fft * wavelet_fft

        # Inverse FFT to get the result back in the time domain
        cwt_matrix[i, :] = ifft(conv_result)

    return cwt_matrix

# Example usage:
# Generating a simple signal: a sum of sine waves with different frequencies
t = np.linspace(0, 1, 1000, False)
signal = np.sin(2 * np.pi * 7 * t) + np.sin(2 * np.pi * 14 * t)

# Define scales for CWT
scales = np.arange(1, 100)

# Compute the fast CWT
cwt_result = fast_cwt(signal, scales)

# Plotting the result (scalogram)
plt.imshow(np.abs(cwt_result), extent=[0, 1, 1, 100], cmap='jet', aspect='auto')
plt.colorbar(label='Magnitude')
plt.title('Continuous Wavelet Transform (CWT) - Scalogram')
plt.ylabel('Scale')
plt.xlabel('Time')
plt.show()
